In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import anndata
import scvi
import scanpy as sc
import matplotlib
import csv
import os
import seaborn as sns
from pySankey.sankey import sankey


from matplotlib.patches import Patch

seed=10
os.chdir(".../Atlas/objects")
sc.logging.print_versions()
!pip list

In [ ]:
#new adata 6.6

adata = sc.read("Atlas6.6_3000HVG_integrated_V2.h5ad")


In [ ]:
model = scvi.model.SCVI.load(".../Atlas/Models/Model_Atlas6.1_V2", adata=adata, use_gpu=False)

In [ ]:
sc.settings.set_figure_params(dpi=200, frameon=False)
sc.set_figure_params(dpi=200)
sc.set_figure_params(figsize=(4, 4))
plt.rcParams['figure.dpi'] = 200
plt.rcParams['figure.figsize'] = (4, 4)


#wd for figures
os.chdir(".../Atlas/Figures/Atlas6")


In [ ]:
print(adata.obs.annotation_final_level1.unique().tolist())

In [ ]:
print(adata.obs.annotation_final_level2.unique().tolist())

In [ ]:
adata.obs['annotation_visualization'] = adata.obs['annotation_final_level1B']


In [ ]:
sc.tl.dendrogram(adata, groupby="leiden_scVI_8_0")



In [ ]:
#we remove the "useless" clusters containing less than 7 cells for visualization

# Filter cells with less than 7 cells in each cluster
min_cells_per_cluster = 10
cell_counts = adata.obs["leiden_scVI_8_0"].value_counts()
clusters_to_keep = cell_counts[cell_counts >= min_cells_per_cluster].index.tolist()
adata = adata[adata.obs["leiden_scVI_8_0"].isin(clusters_to_keep)]

# Recalculate cell counts per cluster
sc.pl.umap(adata, color="leiden_scVI_8_0", legend_loc="on data", title="Clusters (pre-filtering)")
adata.obs["leiden_scVI_8_0"].value_counts()

In [ ]:
sc.tl.dendrogram(adata, groupby="leiden_scVI_8_0")

In [ ]:
plt.rcParams['figure.figsize'] = (4, 40)
sc.pl.dendrogram(adata, groupby = "leiden_scVI_8_0", orientation='left')
plt.rcParams['figure.figsize'] = (4, 4)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

# Get the unique cell types in "leiden_scVI_8_0"
cell_types = adata.obs["leiden_scVI_8_0"].unique()

# Get the cell type order based on the "categories_ordered" key and reverse it
cell_types_sorted = list(reversed(adata.uns["dendrogram_leiden_scVI_8_0"]["categories_ordered"]))


# Define the colors for the different species categories
colors = {"mouse": "C1",  # orange
          "human": "C0",  # blue
          "rat": "C2"}    # green

# Create a figure with a single subplot
fig, ax = plt.subplots(figsize=(32, 3))

# Set the width of each bar and the space between the bars
bar_width = 0.25
bar_space = 0.1

# Loop over each cell type and create a stacked bar plot of the "species" category
for i, cell_type in enumerate(cell_types_sorted):
    cell_type_obs = adata.obs[adata.obs["leiden_scVI_8_0"] == cell_type]
    N = len(cell_type_obs)
    species_counts = cell_type_obs["species"].value_counts()
    species_percent = [species_counts.get(species, 0) / N * 100 for species in colors.keys()]
    bottom = [0]*len(colors)
    for j, species in enumerate(colors.keys()):
        if j == 0:
            bottom[j] = 0
        else:
            bottom[j] = bottom[j-1] + species_percent[j-1]
    ax.bar(i*(bar_width+bar_space), species_percent, bottom=bottom, width=bar_width, color=list(colors.values()))

# Create the legend with colored patches for each species
legend_patches = [Patch(facecolor=color, label=species) for species, color in colors.items()]


# Set the x-axis limits, ticks, and labels, and the y-axis label and title
ax.set_xlim(-bar_space, len(cell_types)*(bar_width+bar_space))
ax.set_xticks([i*(bar_width+bar_space) + (bar_width+bar_space)/2 for i in range(len(cell_types))])
ax.set_xticklabels(cell_types_sorted, rotation=45, ha="right")
ax.set_xlabel("leiden_scVI_8_0")
ax.set_ylabel("Fraction of cells (%)")
ax.set_title("Species composition")

# Save the figure and show the plot
plt.tight_layout()
#plt.savefig("species_composition.png")
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

# Get the unique cell types in "leiden_scVI_8_0"
cell_types = adata.obs["leiden_scVI_8_0"].unique()

# Get the cell type order based on the "categories_ordered" key and reverse it
cell_types_sorted = list(reversed(adata.uns["dendrogram_leiden_scVI_8_0"]["categories_ordered"]))


# Define the colors for the different species categories
colors = {"diseased": "C8",  # orange
          "healthy": "C6",  # blue
         }   

# Create a figure with a single subplot
fig, ax = plt.subplots(figsize=(32, 3))

# Set the width of each bar and the space between the bars
bar_width = 0.25
bar_space = 0.1

# Loop over each cell type and create a stacked bar plot of the "species" category
for i, cell_type in enumerate(cell_types_sorted):
    cell_type_obs = adata.obs[adata.obs["leiden_scVI_8_0"] == cell_type]
    N = len(cell_type_obs)
    species_counts = cell_type_obs["disease"].value_counts()
    species_percent = [species_counts.get(species, 0) / N * 100 for species in colors.keys()]
    bottom = [0]*len(colors)
    for j, species in enumerate(colors.keys()):
        if j == 0:
            bottom[j] = 0
        else:
            bottom[j] = bottom[j-1] + species_percent[j-1]
    ax.bar(i*(bar_width+bar_space), species_percent, bottom=bottom, width=bar_width, color=list(colors.values()))

# Create the legend with colored patches for each species
legend_patches = [Patch(facecolor=color, label=species) for species, color in colors.items()]


# Set the x-axis limits, ticks, and labels, and the y-axis label and title
ax.set_xlim(-bar_space, len(cell_types)*(bar_width+bar_space))
ax.set_xticks([i*(bar_width+bar_space) + (bar_width+bar_space)/2 for i in range(len(cell_types))])
ax.set_xticklabels(cell_types_sorted, rotation=45, ha="right")
ax.set_xlabel("leiden_scVI_8_0")
ax.set_ylabel("Fraction of cells (%)")
ax.set_title("Diseased")

# Save the figure and show the plot
plt.tight_layout()
#plt.savefig("species_composition.png")
plt.show()

In [ ]:


# Get the unique cell types in "leiden_scVI_8_0"
cell_types = adata.obs["leiden_scVI_8_0"].unique()

# Get the cell type order based on the "categories_ordered" key and reverse it
cell_types_sorted = list(reversed(adata.uns["dendrogram_leiden_scVI_8_0"]["categories_ordered"]))


# Define the colors for the different species categories
colors = {"CD_PC": "#c6dbef", 
               "CNT": "#bcbddc", 
               "DCT": "#3182bd",
               "DCT2": "mediumturquoise", 
               "DTL_ATL": "#fdd0a2",
               "EC": "seagreen", 
               "ICA": "orange",
               "ICB": "red", 
               "Immune": "#c7e9c0",
               "Podo": "#000004", 
               "Stromal": "limegreen",
               "PEC": "#fde725",
               "PTS1": "royalblue", 
               "PTS2": "navy",
               "PTS3": "darkcyan", 
               "injPT": "darkorchid",
               "prolif_Tubule": "darkred", 
               "MD": "crimson", 
               "TAL": "lightcoral",
               "injTAL": "IndianRed",
               "injDCT_CNT": "midnightblue",
          }

# Create a figure with a single subplot
fig, ax = plt.subplots(figsize=(32, 3))

# Set the width of each bar and the space between the bars
bar_width = 0.25
bar_space = 0.1

# Loop over each cell type and create a stacked bar plot of the "species" category
for i, cell_type in enumerate(cell_types_sorted):
    cell_type_obs = adata.obs[adata.obs["leiden_scVI_8_0"] == cell_type]
    N = len(cell_type_obs)
    species_counts = cell_type_obs["annotation_final_level1B"].value_counts()
    species_percent = [species_counts.get(species, 0) / N * 100 for species in colors.keys()]
    bottom = [0]*len(colors)
    for j, species in enumerate(colors.keys()):
        if j == 0:
            bottom[j] = 0
        else:
            bottom[j] = bottom[j-1] + species_percent[j-1]
    ax.bar(i*(bar_width+bar_space), species_percent, bottom=bottom, width=bar_width, color=list(colors.values()))

# Create the legend with colored patches for each species
legend_patches = [Patch(facecolor=color, label=species) for species, color in colors.items()]


# Set the x-axis limits, ticks, and labels, and the y-axis label and title
ax.set_xlim(-bar_space, len(cell_types)*(bar_width+bar_space))
ax.set_xticks([i*(bar_width+bar_space) + (bar_width+bar_space)/2 for i in range(len(cell_types))])
ax.set_xticklabels(cell_types_sorted, rotation=45, ha="right")
ax.set_xlabel("leiden_scVI_8_0")
ax.set_ylabel("Fraction of cells (%)")
ax.set_title("Cell Types")

# Save the figure and show the plot
plt.tight_layout()
#plt.savefig("species_composition.png")
plt.show()

In [ ]:


# Define the colors for the different species categories
colors = {"CD_PC": "#c6dbef", 
               "CNT": "#bcbddc", 
               "DCT": "#3182bd",
               "DCT2": "mediumturquoise", 
               "DTL_ATL": "#fdd0a2",
               "EC": "seagreen", 
               "ICA": "orange",
               "ICB": "red", 
               "Immune": "#c7e9c0",
               "Podo": "#000004", 
               "Stromal": "limegreen",
               "PEC": "#fde725",
               "PTS1": "royalblue", 
               "PTS2": "navy",
               "PTS3": "darkcyan", 
               "injPT": "darkorchid",
               "prolif_Tubule": "darkred", 
               "MD": "crimson", 
               "TAL": "lightcoral",
               "injTAL": "IndianRed",
               "injDCT_CNT": "midnightblue",
          }

# Create a figure with a single subplot and add the legend to it
fig, ax = plt.subplots()
legend_patches = [Patch(facecolor=color, label=species) for species, color in colors.items()]
ax.legend(handles=legend_patches, loc="center")

# Hide the axis ticks and labels
ax.axis('off')

# Save the figure and show the plot
plt.tight_layout()

plt.show()

In [ ]:


# Define the colors for the different species categories
colors = {"mouse": "C1",  # orange
          "human": "C0",  # blue
          "rat": "C2"}    # green

# Create a figure with a single subplot and add the legend to it
fig, ax = plt.subplots()
legend_patches = [Patch(facecolor=color, label=species) for species, color in colors.items()]
ax.legend(handles=legend_patches, loc="center")

# Hide the axis ticks and labels
ax.axis('off')

# Save the figure and show the plot
plt.tight_layout()

plt.show()


In [ ]:


# Define the colors for the different species categories
colors = {"diseased": "C8",  # orange
          "healthy": "C6",  # blue
         } 

# Create a figure with a single subplot and add the legend to it
fig, ax = plt.subplots()
legend_patches = [Patch(facecolor=color, label=species) for species, color in colors.items()]
ax.legend(handles=legend_patches, loc="center")

# Hide the axis ticks and labels
ax.axis('off')

# Save the figure and show the plot
plt.tight_layout()

plt.show()

In [ ]:
cell_types_sorted = list(reversed(adata.uns["dendrogram_leiden_scVI_8_0"]["categories_ordered"]))
len(cell_types_sorted)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

# Get the unique cell types in "leiden_scVI_8_0"
cell_types = adata.obs["leiden_scVI_8_0"].unique()

# Get the cell type order based on the "categories_ordered" key and reverse it
cell_types_sorted = list(reversed(adata.uns["dendrogram_leiden_scVI_8_0"]["categories_ordered"]))


# Define the colors for the different species categories
colors = {
# sample color keys
}  

# Create a figure with a single subplot
fig, ax = plt.subplots(figsize=(32, 3))

# Set the width of each bar and the space between the bars
bar_width = 0.25
bar_space = 0.1

# Loop over each cell type and create a stacked bar plot of the "species" category
for i, cell_type in enumerate(cell_types_sorted):
    cell_type_obs = adata.obs[adata.obs["leiden_scVI_8_0"] == cell_type]
    N = len(cell_type_obs)
    species_counts = cell_type_obs["orig_ident"].value_counts()
    species_percent = [species_counts.get(species, 0) / N * 100 for species in colors.keys()]
    bottom = [0]*len(colors)
    for j, species in enumerate(colors.keys()):
        if j == 0:
            bottom[j] = 0
        else:
            bottom[j] = bottom[j-1] + species_percent[j-1]
    ax.bar(i*(bar_width+bar_space), species_percent, bottom=bottom, width=bar_width, color=list(colors.values()))

# Create the legend with colored patches for each species
legend_patches = [Patch(facecolor=color, label=species) for species, color in colors.items()]


# Set the x-axis limits, ticks, and labels, and the y-axis label and title
ax.set_xlim(-bar_space, len(cell_types)*(bar_width+bar_space))
ax.set_xticks([i*(bar_width+bar_space) + (bar_width+bar_space)/2 for i in range(len(cell_types))])
ax.set_xticklabels(cell_types_sorted, rotation=45, ha="right")
ax.set_xlabel("leiden_scVI_8_0")
ax.set_ylabel("Fraction of cells (%)")
ax.set_title("Ident")

# Save the figure and show the plot
plt.tight_layout()
#plt.savefig("species_composition.png")
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

# Get the unique cell types in "leiden_scVI_8_0"
cell_types = adata.obs["leiden_scVI_8_0"].unique()

# Get the cell type order based on the "categories_ordered" key and reverse it
cell_types_sorted = list(reversed(adata.uns["dendrogram_leiden_scVI_8_0"]["categories_ordered"]))


# Define the colors for the different species categories
colors = {
"male": "C12", 
"female": "C13"
}  

# Create a figure with a single subplot
fig, ax = plt.subplots(figsize=(32, 3))

# Set the width of each bar and the space between the bars
bar_width = 0.25
bar_space = 0.1

# Loop over each cell type and create a stacked bar plot of the "species" category
for i, cell_type in enumerate(cell_types_sorted):
    cell_type_obs = adata.obs[adata.obs["leiden_scVI_8_0"] == cell_type]
    N = len(cell_type_obs)
    species_counts = cell_type_obs["sex"].value_counts()
    species_percent = [species_counts.get(species, 0) / N * 100 for species in colors.keys()]
    bottom = [0]*len(colors)
    for j, species in enumerate(colors.keys()):
        if j == 0:
            bottom[j] = 0
        else:
            bottom[j] = bottom[j-1] + species_percent[j-1]
    ax.bar(i*(bar_width+bar_space), species_percent, bottom=bottom, width=bar_width, color=list(colors.values()))

# Create the legend with colored patches for each species
legend_patches = [Patch(facecolor=color, label=species) for species, color in colors.items()]


# Set the x-axis limits, ticks, and labels, and the y-axis label and title
ax.set_xlim(-bar_space, len(cell_types)*(bar_width+bar_space))
ax.set_xticks([i*(bar_width+bar_space) + (bar_width+bar_space)/2 for i in range(len(cell_types))])
ax.set_xticklabels(cell_types_sorted, rotation=45, ha="right")
ax.set_xlabel("leiden_scVI_8_0")
ax.set_ylabel("Fraction of cells (%)")
ax.set_title("Sex")

# Save the figure and show the plot
plt.tight_layout()
#plt.savefig("species_composition.png")
plt.show()

In [ ]:
#only humans


import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

# Get the unique cell types in "leiden_scVI_8_0"
cell_types = adata.obs["leiden_scVI_8_0"].unique()

# Get the cell type order based on the "categories_ordered" key and reverse it
cell_types_sorted = list(reversed(adata.uns["dendrogram_leiden_scVI_8_0"]["categories_ordered"]))


# Define the colors for the different species categories
colors = {
"male": "C12", 
"female": "C13"
}  

# Create a figure with a single subplot
fig, ax = plt.subplots(figsize=(32, 3))

# Set the width of each bar and the space between the bars
bar_width = 0.25
bar_space = 0.1

# Loop over each cell type and create a stacked bar plot of the "species" category
for i, cell_type in enumerate(cell_types_sorted):
    cell_type_obs = adata_human.obs[adata_human.obs["leiden_scVI_8_0"] == cell_type]
    N = len(cell_type_obs)
    species_counts = cell_type_obs["sex"].value_counts()
    species_percent = [species_counts.get(species, 0) / N * 100 for species in colors.keys()]
    bottom = [0]*len(colors)
    for j, species in enumerate(colors.keys()):
        if j == 0:
            bottom[j] = 0
        else:
            bottom[j] = bottom[j-1] + species_percent[j-1]
    ax.bar(i*(bar_width+bar_space), species_percent, bottom=bottom, width=bar_width, color=list(colors.values()))

# Create the legend with colored patches for each species
legend_patches = [Patch(facecolor=color, label=species) for species, color in colors.items()]


# Set the x-axis limits, ticks, and labels, and the y-axis label and title
ax.set_xlim(-bar_space, len(cell_types)*(bar_width+bar_space))
ax.set_xticks([i*(bar_width+bar_space) + (bar_width+bar_space)/2 for i in range(len(cell_types))])
ax.set_xticklabels(cell_types_sorted, rotation=45, ha="right")
ax.set_xlabel("leiden_scVI_8_0")
ax.set_ylabel("Fraction of cells (%)")
ax.set_title("Sex")

# Save the figure and show the plot
plt.tight_layout()
#plt.savefig("species_composition.png")
plt.show()

In [ ]:
#only humans and only KPMP
adata_human = adata_human[adata_human.obs.proj == "h_KPMP"]


import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

# Get the unique cell types in "leiden_scVI_8_0"
cell_types = adata.obs["leiden_scVI_8_0"].unique()

# Get the cell type order based on the "categories_ordered" key and reverse it
cell_types_sorted = list(reversed(adata.uns["dendrogram_leiden_scVI_8_0"]["categories_ordered"]))


# Define the colors for the different species categories
colors = {
    'dPT': '#FF0000', 'aPT': '#00FF00', 'M-FIB': '#0000FF', 'MD': '#FFFF00', 'NKT': '#FF00FF',
    'tPC-IC': '#00FFFF', 'EC-DVR': '#800000', 'CCD-IC-A': '#008000', 'EC-AVR': '#000080', 'MAC-M2': '#808000',
    'cycPT': '#800080', 'OMCD-IC-A': '#008080', 'aTAL1': '#808080', 'EC-AEA': '#FFA500', 'DTL2': '#ADFF2F',
    'M-TAL': '#00BFFF', 'N': '#FF4500', 'POD': '#CD5C5C', 'ATL': '#FFD700', 'DTL1': '#FF69B4',
    'EC-PTC': '#BC8F8F', 'PT-S3': '#7FFF00', 'PT-S1': '#D2691E', 'C-TAL': '#DC143C', 'OMCD-PC': '#8A2BE2',
    'FIB': '#7FFFD4', 'dOMCD-PC': '#C0C0C0', 'T': '#A52A2A', 'PL': '#9400D3', 'CNT-PC': '#228B22',
    'DCT1': '#FF6347', 'CCD-PC': '#40E0D0', 'VSMC': '#20B2AA', 'DTL3': '#9932CC', 'dM-TAL': '#EE82EE',
    'EC-GC': '#8B4513', 'VSMC/P': '#6A5ACD', 'cycMNP': '#9370DB', 'MYOF': '#F08080', 'cycEC': '#FFC0CB',
    'cDC': '#48D1CC', 'EC-LYM': '#6495ED', 'dPT/DTL': '#2E8B57', 'DCT2': '#7B68EE', 'MC': '#32CD32',
    'IC-B': '#FFDAB9', 'MDC': '#4B0082', 'CNT': '#B22222', 'dVSMC': '#2F4F4F', 'ncMON': '#00CED1',
    'dATL': '#DEB887', 'REN': '#BDB76B', 'B': '#66CDAA', 'aTAL2': '#F5DEB3', 'dFIB': '#FFFAF0',
    'dC-IC-A': '#FA8072', 'MAST': '#8B0000', 'aFIB': '#00FA9A', 'PEC': '#F5F5DC', 'dCNT': '#3CB371',
    'cycCNT': '#87CEEB', 'dIMCD': '#ADD8E6', 'PT-S2': '#F0E68C', 'dC-TAL': '#E0FFFF', 'dEC-PTC': '#87CEFA',
    'dEC': '#D2B48C', 'dM-FIB': '#FFA07A', 'CNT-IC-A': '#4682B4', 'SC/NEU': '#9ACD32', 'pDC': '#EE82EE', 'IMCD': '#556B2F', 'dPOD': '#5F9EA0', 'dDTL3': '#8B008B',
    'cycDCT': '#483D8B', 'dDCT': '#00FFFF', 'cycMYOF': '#1E90FF', 'PapE': '#B0E0E6'
}





 

# Create a figure with a single subplot
fig, ax = plt.subplots(figsize=(32, 3))

# Set the width of each bar and the space between the bars
bar_width = 0.25
bar_space = 0.1

# Loop over each cell type and create a stacked bar plot of the "species" category
for i, cell_type in enumerate(cell_types_sorted):
    cell_type_obs = adata_human.obs[adata_human.obs["leiden_scVI_8_0"] == cell_type]
    N = len(cell_type_obs)
    species_counts = cell_type_obs["subclass.l2"].value_counts()
    species_percent = [species_counts.get(species, 0) / N * 100 for species in colors.keys()]
    bottom = [0]*len(colors)
    for j, species in enumerate(colors.keys()):
        if j == 0:
            bottom[j] = 0
        else:
            bottom[j] = bottom[j-1] + species_percent[j-1]
    ax.bar(i*(bar_width+bar_space), species_percent, bottom=bottom, width=bar_width, color=list(colors.values()))

# Create the legend with colored patches for each species
legend_patches = [Patch(facecolor=color, label=species) for species, color in colors.items()]


# Set the x-axis limits, ticks, and labels, and the y-axis label and title
ax.set_xlim(-bar_space, len(cell_types)*(bar_width+bar_space))
ax.set_xticks([i*(bar_width+bar_space) + (bar_width+bar_space)/2 for i in range(len(cell_types))])
ax.set_xticklabels(cell_types_sorted, rotation=45, ha="right")
ax.set_xlabel("leiden_scVI_8_0")
ax.set_ylabel("Fraction of cells (%)")
ax.set_title("KPMP Annotation")

# Save the figure and show the plot
plt.tight_layout()
#plt.savefig("species_composition.png")
plt.show()

In [ ]:
# Define the colors for the different species categories
colors = {
   
    
    
    
    # Proximal Tubule cells
    'dPT': '#FF0000', 'aPT': '#00FF00', 'PT-S1': '#D2691E', 'PT-S2': '#E0FFFF', 'PT-S3': '#7FFF00',
    # Distal Tubule cells
    'DTL1': '#FF69B4', 'DTL2': '#ADFF2F', 'DTL3': '#9932CC', 'dDTL3': '#8B008B', 'dPT/DTL': '#2E8B57',
    # Thick Ascending Limb cells
    'M-TAL': '#00BFFF', 'dM-TAL': '#EE82EE', 'aTAL1': '#808080', 'aTAL2': '#F5DEB3',
    # Thin Ascending Limb cells
    'ATL': '#FFD700', 'dATL': '#DEB887',
    # Collecting Duct cells
    'CCD-IC-A': '#008000', 'CCD-PC': '#40E0D0', 'CNT-IC-A': '#4682B4', 'CNT-PC': '#228B22', 'CNT': '#B22222', 'dCNT': '#3CB371', 'cycCNT': '#87CEEB', 'dIMCD': '#ADD8E6', 'IMCD': '#556B2F',
    # Loop of Henle cells
    'C-TAL': '#DC143C', 'dC-TAL': '#800080', 'DCT1': '#FF6347', 'DCT2': '#7B68EE', 'dDCT': '#00FFFF', 'cycDCT': '#483D8B',
    # Fibroblast cells
    'FIB': '#7FFFD4', 'dFIB': '#FFFAF0', 'M-FIB': '#0000FF', 'dM-FIB': '#FFA07A', 'aFIB': '#00FA9A',
    # Immune cells
    'B': '#66CDAA', 'MC': '#32CD32', 'MAST': '#8B0000', 'MD': '#FFFF00', 'MDC': '#4B0082', 'NKT': '#FF00FF', 'ncMON': '#00CED1', 'REN': '#BDB76B',
    # Endothelial cells
    'EC-AEA': '#FFA500', 'EC-AVR': '#000080', 'EC-DVR': '#800000', 'EC-GC': '#8B4513', 'EC-LYM': '#6495ED', 'EC-PTC': '#BC8F8F', 'dEC': '#D2B48C', 'dEC-PTC': '#87CEFA', 'cycEC': '#FFC0CB',
    # Interstitial cells
    'IC-B': '#FFDAB9', 'dC-IC-A': '#FA8072',
    # Myofibroblast cells
    'MYOF': '#F08080', 'cycMYOF': '#1E90FF',
    # Vascular Smooth Muscle cells
    'VSMC': '#20B2AA', 'dVSMC': '#2F4F4F', 'VSMC/P': '#6A5ACD',
    # Other cell types
    'cDC': '#48D1CC', 'cycMNP': '#9370DB', 'cycPT': '#808000', 'dOMCD-PC': '#C0C0C0', 'FIB': '#7FFFD4', 'OMCD-IC-A': '#008080',
    'OMCD-PC': '#8A2BE2', 'PEC': '#F5F5DC', 'PL': '#9400D3', 'pDC': '#EE82EE', 'POD': '#CD5C5C', 'SC/NEU': '#9ACD32', 'T': '#A52A2A', 'N': '#228B22', 'PapE': '#B0E0E6'

    
    
}

# Create a figure with a single subplot and add the legend to it
fig, ax = plt.subplots()
legend_patches = [Patch(facecolor=color, label=species) for species, color in colors.items()]
ax.legend(handles=legend_patches, loc="center")

# Hide the axis ticks and labels
ax.axis('off')

# Save the figure and show the plot
plt.tight_layout()

plt.show()

In [ ]:
color_palette = {
    "0": "#00FFFF",
    "1": "#00BFFF",
    "2": "#1E90FF",
    "3": "#4169E1",
    "4": "#0000FF",
    "5": "#00008B",
    "6": "#000080",
    "7": "#191970",
    "8": "#7B68EE",
    "9": "#6A5ACD",
    "10": "#483D8B",
    "11": "#9370DB",
    "12": "#8A2BE2",
    "13": "#9400D3",
    "14": "#4B0082",
    "15": "#BA55D3",
    "16": "#FF00FF",
    "17": "#8B008B",
    "18": "#800080",
    "19": "#C71585",
    "20": "#FF1493",
    "21": "#FF69B4",
    "22": "#FFB6C1",
    "23": "#FFC0CB",
    "24": "#DC143C",
    "25": "#FF0000",
    "26": "#B22222",
    "27": "#8B0000",
    "28": "#FF7F50",
    "29": "#FF6347",
    "30": "#FF4500",
    "31": "#FF8C00",
    "32": "#FFA500",
    "33": "#FFFF00",
    "34": "#FFD700",
    "35": "#DAA520",
    "36": "#ADFF2F",
    "37": "#32CD32",
    "38": "#008000",
    "39": "#006400",
    "40": "#7FFF00",
    "41": "#00FF00",
    "42": "#00FA9A",
    "43": "#00FF7F",
    "44": "#66CDAA",
    "45": "#20B2AA",
    "46": "#008B8B",
    "47": "#008080",
    "48": "#00FFFF",
    "49": "#E0FFFF",
    "50": "#AFEEEE",
    "51": "#7FFFD4",
    "52": "#40E0D0",
    "53": "#48D1CC",
    "54": "#00CED1",
    "55": "#5F9EA0",
    "56": "#4682B4",
    "57": "#B0C4DE",
    "58": "#ADD8E6",
    "59": "#87CEFA",
    "60": "#87CEEB",
    "61": "#6495ED",
    "62": "#00BFFF",
    "63": "#1E90FF",
    "64": "#4169E1",
    "65": "#0000FF",
    "66": "#00008B",
    "67": "#000080",
    "68": "#191970",
    "69": "#7B68EE",
    "70": "#6A5ACD",
    "71": "#483D8B",
    "72": "#9370DB",
    "73": "#8A2BE2",
    "74": "#9400D3",
    "75": "#4B0082",
    "76": "#BA55D3",
    "77": "#FF00FF",
    "78": "red",
    "79": "#8B008B",
    "80": "#800080",
    "81": "#C71585",
    "82": "#FF1493",
    "83": "#FF69B4",
    "84": "#FFB6C1",
    "85": "#FFC0CB",
    "86": "#DC143C",
    "87": "#FF0000",
    "88": "#B22222",
    "89": "#8B0000",
    "90": "#FF7F50",
    "91": "#FF6347",
    "92": "#FF4500",
    "93": "#FF8C00",
    "94": "#FFA500",
    "95": "#FFFF00",
    "96": "#FFD700",
    "97": "#DAA520",
    "98": "#ADFF2F",
    "99": "#32CD32",
    "100": "#008000",
    "101": "#006400",
    "102": "#7FFF00",
    "103": "#00FF00",
    "104": "#00FA9A",
    "105": "#00FF7F",
    "106": "#66CDAA",
    "107": "#20B2AA",
    "108": "#008B8B",
    "109": "#008080",
    "110": "#00FFFF",
    "111": "#E0FFFF",
    "112": "#AFEEEE",
    "113": "#7FFFD4",
    "114": "#40E0D0",
    "115": "#48D1CC",
    "116": "#00CED1",
    "117": "#5F9EA0",
    "118": "#4682B4",
    "119": "#B0C4DE",
    "120": "#ADD8E6",
    "121": "#87CEFA",
    "122": "#87CEEB",
    "123": "#6495ED",
    "124": "#00BFFF",
    "125": "#1E90FF",
    "126": "#4169E1",
    "127": "#0000FF",
    "128": "#00008B",
    "129": "#000080",
    "130": "#191970",
    "131": "#7B68EE",
    "132": "#6A5ACD",
    "133": "#483D8B",
    "134": "#9370DB",
    "135": "#8A2BE2",
    "136": "#9400D3",
    "137": "#4B0082",
    "138": "#BA55D3",
    "139": "#FF00FF",
    "140": "#8B008B",
    "141": "#800080",
    "142": "#C71585",
    "143": "#FF1493",
    "144": "#FF69B4",
    "145": "#FFB6C1",
    "146": "#FFC0CB",
    "147": "#DC143C",
    "148": "#FF0000",
    "149": "#B22222",
    "150": "#8B0000",
    "151": "#FF7F50",
    "152": "#FF6347",
    "153": "#FF4500",
    "154": "#FF8C00",
    "155": "#FFA500",
    "156": "#FFFF00",
    "157": "#FFD700",
    "158": "#DAA520",
    "159": "#ADFF2F",
    "160": "#32CD32",
    "161": "#008000",
    "162": "#006400",
    "163": "#7FFF00",
    "164": "#00FF00",
    "165": "#00FA9A",
    "166": "#00FF7F",
    "167": "#66CDAA",
    "168": "#20B2AA",
    "169": "#008B8B",
    "170": "#008080",
    "171": "#00FFFF",
    "172": "#E0FFFF",
    "173": "#AFEEEE",
    "174": "#7FFFD4",
    "175": "#40E0D0",
    "176": "#48D1CC",
    "177": "#00CED1",
    "178": "#5F9EA0",
    "179": "#4682B4",
    "180": "#B0C4DE",
    "181": "#ADD8E6",
    "182": "#87CEFA",
    "183": "#87CEEB",
    "184": "#6495ED",
    "185": "#00BFFF",
    "186": "#1E90FF",
    "187": "#4169E1",
    "188": "#0000FF",
    "189": "#00008B",
    "190": "#000080",
    "191": "#191970",
    "192": "#7B68EE",
    "193": "#6A5ACD",
    "194": "#483D8B",
    "195": "#9370DB",
    "196": "#8A2BE2",
    "197": "#9400D3",
    "198": "#4B0082",
    "199": "#BA55D3",
}

In [ ]:
sc.pl.umap(adata, color=["leiden_scVI_8_0"], palette = color_palette, frameon=False, title="", legend_loc = False, add_outline=True)



In [ ]:
sc.settings.set_figure_params(dpi=200, frameon=False)
sc.set_figure_params(dpi=200, fontsize=4, figsize=(4, 4))
plt.rcParams['figure.dpi'] = 200
plt.rcParams['figure.figsize'] = (4, 4)



sc.pl.umap(adata, color=["leiden_scVI_8_0"], palette = color_palette, frameon=False, title="", legend_loc = "on data", add_outline=True)

